In [1]:
import tensorflow as tf
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cPickle as pickle
import h5py
import random

In [2]:
class MTGRU_precode(object):

    def init_weights(self, input_dim, output_dim, name=None, std=1.0):
        return tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=std / math.sqrt(input_dim)), name=name)

    def init_bias(self, output_dim, name=None):
        return tf.Variable(tf.zeros([output_dim]), name=name)

    def __init__(self, visit_num, visit_length, one_hot_input_dim, input_dim, info_dim, output_dim1,
                 output_dim2, voutput_dim,output_dim, vhidden_dim, hidden_dim1, hidden_dim2,patient_visit_num,patient_visit_length,visit_time,visit_info,patient_num,code_inputindex,code_time):

        #参数patient_visit_num, patient_visit_length, visit_time, visit_info, patient_num, code_inputindex, code_time用来对新的 patient预测疾病
        # 一个病人visit的个数
        self.visit_num = visit_num
        # 一个visit中的code的个数
        self.visit_length = visit_length

        # 输入的one-hot的维度
        self.one_hot_input_dim = one_hot_input_dim
        # one-hot变换后作为网络输入的维度
        self.input_dim = input_dim
        # 患者信息
        self.info_dim = info_dim


        # 中间层输出,每层gru有一个输出，visit层有一个输出
        self.output_dim1 = output_dim1
        self.output_dim2 = output_dim2
        self.voutput_dim = voutput_dim
        # 中间隐藏层，四个gru层，一个visit层
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.vhidden_dim = vhidden_dim

        # 最后一层gru最终输出
        self.output_dim = output_dim
        final_output_dim=one_hot_input_dim
        self.final_output_dim=one_hot_input_dim

        # one-hot后的矩阵参数
        self.Wi = self.init_weights(one_hot_input_dim, input_dim, name='OneHot_w')
        self.bi = self.init_bias(input_dim, name='OneHot_w')

        # encoder的GRU参数
        self.Wz_enc = self.init_weights(input_dim, hidden_dim1, name='Update_wx_enc')
        self.Uz_enc = self.init_weights(hidden_dim1, hidden_dim1, name='Update_wh_enc')
        self.bz_enc = self.init_bias(hidden_dim1, name='Update_bias_enc')

        self.Wz_enc2 = self.init_weights(output_dim1, hidden_dim2, name='Update_wx_enc2')
        self.Uz_enc2 = self.init_weights(hidden_dim2, hidden_dim2, name='Update_wh_enc2')
        self.bz_enc2 = self.init_bias(hidden_dim2, name='Update_bias_enc2')

        self.Wr_enc = self.init_weights(input_dim, hidden_dim1, name='Reset_wx_enc')
        self.Ur_enc = self.init_weights(hidden_dim1, hidden_dim1, name='Reset_wh_enc')
        self.br_enc = self.init_bias(hidden_dim1, name='Reset_bias_enc')

        self.Wr_enc2 = self.init_weights(output_dim1, hidden_dim2, name='Reset_wx_enc2')
        self.Ur_enc2 = self.init_weights(hidden_dim2, hidden_dim2, name='Reset_wh_enc2')
        self.br_enc2 = self.init_bias(hidden_dim2, name='Reset_bias_enc2')

        self.Wd_enc = tf.ones([1, self.hidden_dim1], dtype=tf.float32, name='Decay_w_enc')
        self.Wd_enc2 = tf.ones([1, self.hidden_dim2], dtype=tf.float32, name='Decay_w_enc2')

        self.Wh_enc = self.init_weights(self.input_dim, self.hidden_dim1, name='Canditateh_wx_enc')
        self.Uh_enc = self.init_weights(self.hidden_dim1, self.hidden_dim1, name='Canditateh_wh_enc')
        self.bh_enc = self.init_bias(self.hidden_dim1, name='Canditateh_bias_enc')

        self.Wh_enc2 = self.init_weights(self.output_dim1, self.hidden_dim2, name='Canditateh_wx_enc2')
        self.Uh_enc2 = self.init_weights(self.hidden_dim2, self.hidden_dim2, name='Canditateh_wh_enc2')
        self.bh_enc2 = self.init_bias(self.hidden_dim2, name='Canditateh_bias_enc2')


        # visit层的参数
        # 输入是上一层的输出加上病人信息
        self.Wz_v = self.init_weights(output_dim2 + info_dim, vhidden_dim, name='Update_wx_v')
        self.Uz_v = self.init_weights(vhidden_dim, vhidden_dim, name='Update_wh_v')
        self.bz_v = self.init_bias(vhidden_dim, name='Update_bias_dec')

        self.Wr_v = self.init_weights(output_dim2 + info_dim, vhidden_dim, name='Reset_wx_v')
        self.Ur_v = self.init_weights(vhidden_dim, vhidden_dim, name='Reset_wh_v')
        self.br_v = self.init_bias(vhidden_dim, name='Reset_bias_v')

        self.Wd_v = tf.ones([1, self.vhidden_dim], dtype=tf.float32, name='Decay_w_v')

        self.Wh_v = self.init_weights(self.output_dim2 + info_dim, self.vhidden_dim, name='Canditateh_wx_v')
        self.Uh_v = self.init_weights(self.vhidden_dim, self.vhidden_dim, name='Canditateh_wh_v')
        self.bh_v = self.init_bias(self.vhidden_dim, name='Canditateh_bias_v')


        # 输出层
        # encoder1
        self.Wo1 = self.init_weights(hidden_dim1, output_dim1, name='Output1_weight')
        self.bo1 = self.init_bias(output_dim1, name='Output1_bias')
        # encoder2
        self.Wo2 = self.init_weights(hidden_dim2, output_dim2, name='Output2_weight')
        self.bo2 = self.init_bias(output_dim2, name='Output2_bias')
        # visit层的输出
        self.Wo = self.init_weights(vhidden_dim, output_dim, name='Output_w')
        self.bo = self.init_bias(output_dim, name='Output_bias')
        # 最终输出全连接
        self.Wf = self.init_weights(output_dim, final_output_dim, name='Final_output_w')
        self.bf = self.init_bias(final_output_dim, name='Final_output_bias')


        # 输入占位符
        # [batch size x seq length x input dim]
        self.inputindex=tf.placeholder(dtype=tf.int32,shape=[None,None])
        self.one_hot_input=tf.one_hot(indices=self.inputindex, depth=self.one_hot_input_dim, axis=2) #输入
        self.lable_multi_hot=tf.placeholder('float',shape=[None,final_output_dim]) #标签
        self.time = tf.placeholder('float', [None, None])
        self.keep_prob = tf.placeholder(tf.float32)
        self.info = tf.placeholder('float', shape=[None, None, self.info_dim])# 患者人口信息


        # 对新的人来预测疾病
        self.patient_visit_num = patient_visit_num # 一个患者的visit个数
        self.patient_visit_length = patient_visit_length # 一个患者的visit中的code个数
        self.visit_time = visit_time
        self.visit_info = visit_info
        self.patient_num=patient_num
        self.code_inputindex=code_inputindex #一个病人所有code的index
        self.code_time=code_time



    # 对输入的one-hot使用矩阵处理，code的嵌入向量1
    def get_input(self, one_hot_input):
        input = tf.matmul(one_hot_input, self.Wi) + self.bi
        return input

    # encoder第一层TGRU cell，code的嵌入向量2
    def TGRU_encoder_cell1(self, prev_h, concat_input):

        # concat_input:[batch_size x input_dim+1]
        batch_size = tf.shape(concat_input)[0]
        x = tf.slice(concat_input, [0, 1], [batch_size, self.input_dim])
        t = tf.slice(concat_input, [0, 0], [batch_size, 1])

        ft = self.map_elapse_time(t, self.hidden_dim1)

        z = tf.sigmoid(tf.matmul(x, self.Wz_enc) + tf.matmul(prev_h, self.Uz_enc) + self.bz_enc)
        r = tf.sigmoid(tf.matmul(x, self.Wr_enc) + tf.matmul(prev_h, self.Ur_enc) + self.br_enc)
        d = tf.matmul(ft, self.Wd_enc)
        h_ = tf.multiply(d, prev_h)
        h_canditate = tf.sigmoid(tf.matmul(x, self.Wh_enc) + tf.matmul(r * h_, self.Uh_enc) + self.bh_enc)

        current_h = h_ - z * h_ + z * h_canditate

        return current_h

    # encoder第二层TGRU cell
    def TGRU_encoder_cell2(self, prev_h, concat_input):

        batch_size = tf.shape(concat_input)[0]
        x = tf.slice(concat_input, [0, 1], [batch_size, self.output_dim1])
        t = tf.slice(concat_input, [0, 0], [batch_size, 1])

        ft = self.map_elapse_time(t, self.hidden_dim2)

        z = tf.sigmoid(tf.matmul(x, self.Wz_enc2) + tf.matmul(prev_h, self.Uz_enc2) + self.bz_enc2)
        r = tf.sigmoid(tf.matmul(x, self.Wr_enc2) + tf.matmul(prev_h, self.Ur_enc2) + self.br_enc2)
        d = tf.matmul(ft, self.Wd_enc2)
        h_ = tf.multiply(d, prev_h)
        h_canditate = tf.sigmoid(tf.matmul(x, self.Wh_enc2) + tf.matmul(r * h_, self.Uh_enc2) + self.bh_enc2)
        current_h = h_ - z * h_ + z * h_canditate

        return current_h

    # visit层的TGURU cell
    # 此时的输入与其他cell的不同，concat_input应该是多个时刻的code的，而其他的输入都是一个时刻的code，因此在后面对get_encoder_h2输出的结果做处理
    # 此时的concat_input是一个三维向量，也就是有多时刻的code，每个时刻有一个时间和batch个数个code
    # [visit_length x batch_size x 1+input_dim ]
    # 创建每个visit的长度的参数，来方便创建多个输入x
    def TGRU_visit_cell(self, prev_h, concat_input):

        # visit_length x batch_size x input_dim+info_dim+info_dim+1
        batch_size = tf.shape(concat_input)[1]

        # 只获取第一个时间
        t = tf.slice(concat_input, [0, 0, 0], [1, batch_size, 1])
        # 只获取第一个info
        info = tf.slice(concat_input, [0, 0, 1], [1, batch_size, self.info_dim])

        # 循环创建局部变量
        for i in range(self.visit_length):
            locals()['x' + str(i)] = tf.slice(concat_input, [i, 0, 1 + self.info_dim],
                                              [1, batch_size, self.output_dim2])

        # 获得visit层的输入
        visit_x = tf.zeros([batch_size, self.output_dim2], dtype=tf.float32)
        for i in range(self.visit_length):
            visit_x += locals()['x' + str(i)]

        visit_x = visit_x / self.visit_length

        # 把输入和病人信息放在一个向量中
        visit_x = tf.concat([visit_x, info], 2)
        # 3维变2维 第一维是大小是1，没用
        visit_x = tf.reshape(visit_x,[tf.shape(visit_x)[1],tf.shape(visit_x)[2]])
        t = tf.reshape(t,[tf.shape(t)[1],tf.shape(t)[2]])


        # visit嵌入向量经过GRU输出
        ft = self.map_elapse_time(t, self.vhidden_dim)
        z = tf.sigmoid(tf.matmul(visit_x, self.Wz_v) + tf.matmul(prev_h, self.Uz_v) + self.bz_v)
        r = tf.sigmoid(tf.matmul(visit_x, self.Wr_v) + tf.matmul(prev_h, self.Ur_v) + self.br_v)
        d = tf.matmul(ft, self.Wd_v)
        h_ = tf.multiply(d, prev_h)
        h_canditate = tf.sigmoid(tf.matmul(visit_x, self.Wh_v) + tf.matmul(r * h_, self.Uh_v) + self.bh_v)
        current_h = h_ - z * h_ + z * h_canditate

        return current_h


    # 每个输出层的操作
    # encoder1的输出
    def get_output1(self, h):
        output = tf.matmul(h, self.Wo1) + self.bo1
        # output = tf.nn.softmax(tf.nn.relu(tf.matmul(state, self.Wo) + self.bo))
        return output

    # encoder2的输出
    def get_output2(self, h):
        output = tf.matmul(h, self.Wo2) + self.bo2
        # output = tf.nn.softmax(tf.nn.relu(tf.matmul(state, self.Wo) + self.bo))
        return output

    # 输出层，对h进行全连接输出
    def get_output(self, h):
        output = tf.nn.relu(tf.matmul(h, self.Wo) + self.bo)
        output = tf.nn.dropout(output, self.keep_prob)
        output = tf.matmul(output, self.Wf) + self.bf
        return output


    # encoder1的输出h
    def get_encoder1_h(self):  # Returns all hidden states for the samples in a batch

        convert_input = self.get_input(self.one_hot_input)
        batch_size = tf.shape(convert_input)[0]
        scan_input = tf.transpose(convert_input, perm=[1, 0, 2])  # scan input is [seq_length x batch_size x input_dim]
        scan_time = tf.transpose(self.time)  # scan_time [seq_length x batch_size]

        initial_hidden = tf.zeros([batch_size, self.hidden_dim1], tf.float32)

        # make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])
        concat_input = tf.concat([scan_time, scan_input], 2)  # [seq_length x batch_size x input_dim+1]

        encoder1_h = tf.scan(self.TGRU_encoder_cell1, concat_input, initializer=initial_hidden, name='encoder1_h')

        return encoder1_h

    # encoder2的输出h
    def get_encoder2_h(self):  # Returns all hidden states for the samples in a batch

        encoder1_h = self.get_encoder1_h()
        encoder1_output = tf.map_fn(self.get_output1, encoder1_h)

        batch_size = tf.shape(encoder1_h)[1]
        scan_time = tf.transpose(self.time)  # scan_time [seq_length x batch_size]
        initial_hidden = tf.zeros([batch_size, self.hidden_dim2], tf.float32)

        # make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])
        concat_input = tf.concat([scan_time, encoder1_output], 2)  # [seq_length x batch_size x input_dim+1]

        encoder2_h = tf.scan(self.TGRU_encoder_cell2, concat_input, initializer=initial_hidden, name='encoder2_h')
        return encoder2_h

    # visit最后的输出h
    def get_visit_h(self):  # Returns all hidden states for the samples in a batch

        encoder2_h = self.get_encoder2_h()
        encoder2_output = tf.map_fn(self.get_output2, encoder2_h)

        batch_size = tf.shape(encoder2_h)[1]
        scan_time = tf.transpose(self.time)  # scan_time [seq_length x batch_size]

        # make info [batch_size x seq_length x info_dim] --> [seq_length x batch_size x info_dim]
        scan_info = tf.transpose(self.info, [1, 0, 2])
        # make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])

        # [seq_length x batch_size x input_dim+info_dim+1]
        concat_input = tf.concat([scan_time, scan_info, encoder2_output], 2)

        # 已经知道了visit的个数和长度，可以直接把concat_input拆分成visit个
        # [visit_num x visit_length x batch_size x input_dim+info_dim_1]
        initial_hidden = tf.zeros([ batch_size, self.vhidden_dim], tf.float32)
        concat_input = tf.reshape(concat_input, [self.visit_num, self.visit_length, tf.shape(concat_input)[1],
                                                 tf.shape(concat_input)[2]])
        visit_h = tf.scan(self.TGRU_visit_cell, concat_input, initializer=initial_hidden, name='visit')
        final_h = tf.reverse(visit_h, [0])[0, :, :]
        return final_h

    def get_cross_loss(self):
        output = self.get_visit_h()
        final_output=self.get_output(output)
        cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.lable_multi_hot, logits=final_output))
        
        # 准确度
        pre=tf.sigmoid(final_output)
        dic=tf.abs(self.lable_multi_hot-pre)
        
        return cross_entropy, dic


    def map_elapse_time(self, t, dim):
        c1 = tf.constant(1, dtype=tf.float32)
        c2 = tf.constant(2.7183, dtype=tf.float32)

        T = tf.div(c1, tf.log(t + c2), name='Log_elapse_time')
        # T = tf.div(c1, tf.add(t , c1), name='Log_elapse_time')
        return T
    
    
    
    # 获取数据集中病人的预测
    def TGRU_visit_cell2(self, prev_h, concat_input):
        # batch_size x input_dim+info_dim+info_dim+1
        batch_size = tf.shape(concat_input)[0]

        # 时间
        t = tf.slice(concat_input, [0, 0], [batch_size, 1])
        # visit_info
        info = tf.slice(concat_input, [0, 1], [batch_size, self.info_dim])
        # 输入
        x = tf.slice(concat_input, [0, 1+self.info_dim], [batch_size, self.output_dim2])

        # 把输入和病人信息放在一个向量中
        visit_x = tf.concat([x, info], 1)

        # visit嵌入向量经过GRU输出
        ft = self.map_elapse_time(t, self.vhidden_dim)
        z = tf.sigmoid(tf.matmul(visit_x, self.Wz_v) + tf.matmul(prev_h, self.Uz_v) + self.bz_v)
        r = tf.sigmoid(tf.matmul(visit_x, self.Wr_v) + tf.matmul(prev_h, self.Ur_v) + self.br_v)
        d = tf.matmul(ft, self.Wd_v)
        h_ = tf.multiply(d, prev_h)
        h_canditate = tf.sigmoid(tf.matmul(visit_x, self.Wh_v) + tf.matmul(r * h_, self.Uh_v) + self.bh_v)
        current_h = h_ - z * h_ + z * h_canditate

        return current_h
    # 备注的矩阵大小是假设一个用户的code有39个，2个visit，总共的code种类259个
    def get_encoder1_h_forPre(self,no):  # Returns all hidden states for the samples in a batch
        convert_input = self.get_input(tf.one_hot(indices=self.code_inputindex[no], depth=self.one_hot_input_dim,axis=1))
        batch_size = 1

        # (39, 1, 259)
        scan_input = tf.transpose([convert_input], perm=[1, 0, 2])  # scan input is [seq_length x batch_size x input_dim]
        # (39, 1)
        scan_time = tf.transpose([tf.to_float(self.code_time[no])])  # scan_time [seq_length x batch_size]
        # (1, 150)
        initial_hidden = tf.zeros([batch_size, self.hidden_dim1], tf.float32)
        # (39, 1, 1) make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])
        # (39, 1, 260)
        concat_input = tf.concat([scan_time, scan_input], 2)  # [seq_length x batch_size x input_dim+1]

        encoder1_h = tf.scan(self.TGRU_encoder_cell1, concat_input, initializer=initial_hidden, name='encoder1_h_forVector')
        return encoder1_h

    def get_encoder2_h_forPre(self,no):  # Returns all hidden states for the samples in a batch
        # (39, 1, 150)
        encoder1_h = self.get_encoder1_h_forPre(no)
        # (39, 1, 150)
        encoder1_output = tf.map_fn(self.get_output1, encoder1_h)
        batch_size = 1
        # (39, 1)
        scan_time = tf.transpose([tf.to_float(self.code_time[no])])  # scan_time [seq_length x batch_size]
        # (1, 150)
        initial_hidden = tf.zeros([batch_size, self.hidden_dim2], tf.float32)
        # (39, 1, 1) # make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])
        # (39, 1, 151)
        concat_input = tf.concat([scan_time, encoder1_output], 2)  # [seq_length x batch_size x input_dim+1]

        encoder2_h = tf.scan(self.TGRU_encoder_cell2, concat_input, initializer=initial_hidden, name='encoder2_h_forVector')
        return encoder2_h

    def get_one_visit_Pre(self,no):
        encoder2_h = self.get_encoder2_h_forPre(no)
        encoder2_output = tf.map_fn(self.get_output2, encoder2_h)
        batch_size= 1
        n = 0
        x=[]
        for i in range(self.patient_visit_num[no][0]):
            #(1, 150)
            visit_input=tf.zeros([batch_size, self.output_dim2], dtype=tf.float32)
            for j in range(self.patient_visit_length[no][i]):
                visit_input+=encoder2_output[n]
                n=n+1
            x.append(visit_input/self.patient_visit_length[no][i])
        # (2, 1)
        scan_time = tf.transpose([tf.to_float(self.visit_time[no])])  # scan_time [seq_length x batch_size]
        # (2, 1, 2) make info [batch_size x seq_length x info_dim] --> [seq_length x batch_size x info_dim]
        scan_info = tf.transpose(tf.to_float([self.visit_info[no]]), [1, 0, 2])
        # (2, 1, 1)make scan_time [seq_length x batch_size x 1]
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])
        # (2, 1, 153)[seq_length x batch_size x input_dim+info_dim+1]
        concat_input = tf.concat([scan_time, scan_info, x], 2)
        # (1,100)
        initial_hidden = tf.zeros([batch_size, self.vhidden_dim], tf.float32)

        visit = tf.scan(self.TGRU_visit_cell2, concat_input, initializer=initial_hidden, name='visit')
        visit_final_h = tf.reverse(visit,[0])[0, :, :]
        return encoder2_output, visit_final_h

    def get_one_pre(self, h):
        output = tf.nn.relu(tf.matmul(h, self.Wo) + self.bo)
        pre = tf.matmul(output, self.Wf) + self.bf
        return pre
    

    def get_all_Pre_sigmoid(self):
        pres=[]
        attentions=[]
        for no in range(self.patient_num):
            encoder2_output , visit_final_h =self.get_one_visit_Pre(no)
            attentions.append(encoder2_output)
            pre=tf.sigmoid(self.get_one_pre(visit_final_h))
            pres.append(pre)
        return pres,attentions


In [3]:
# batch生成器
# bacth中有cutting和padding，由于长度不同，因此开始学习的时候可以每个数据单独学习
# 生成的batch里的病人visit和code的个数一样
def batch_generator(outFile, visit_num, code_num, patient_num, batch_size, icd9_num):
    patient_code_file=open(outFile + '/patient_code' + '.seqs','rb')
    patient_code=pickle.load(patient_code_file)
    
    patient_visit_code_file=open(outFile + '/newSeqs' + '.seqs','rb')
    patient_visit_code=pickle.load(patient_visit_code_file)
    
    codespatientsinfo_file=open(outFile + '/codespatientsinfo' + '.seqs','rb')
    codespatientsinfo=pickle.load(codespatientsinfo_file)
    
    visit_delt_dates_file=open(outFile + '/visit_delt_dates' + '.seqs','rb')
    visit_delt_dates=pickle.load(visit_delt_dates_file)
    
    code_delt_dates_file=open(outFile + '/code_delt_dates' + '.seqs','rb')
    code_delt_dates=pickle.load(code_delt_dates_file)
    
    visits_num_file=open(outFile + '/visits_num' + '.seqs','rb')
    visits_num=pickle.load(visits_num_file)
    
    codes_num_file=open(outFile + '/codes_num' + '.seqs','rb')
    codes_num=pickle.load(codes_num_file)
    
    patient_code_file.close()
    patient_visit_code_file.close()
    codespatientsinfo_file.close()
    visit_delt_dates_file.close()
    code_delt_dates_file.close()
    visits_num_file.close()
    codes_num_file.close()

    
    batch_patient_code=[]
    batch_code_delt_dates=[]
    batch_codespatientsinfo=[]
    batch_lable=[]
    batch_lable_multi_hot=[]
    
    # padding and cutting
    for i in range(batch_size):
        j=random.randint(0, patient_num-1)
        if visits_num[j][0]== visit_num:
            #print 'pick',j
            code=[]
            code_date=[]
            code_info=[]
            for k in range(visit_num-1):
                if  codes_num[j][k]> code_num:
                    if k==0: 
                        code.extend(patient_code[j][0:code_num])
                        code_date.extend(code_delt_dates[j][0:code_num])
                        code_info.extend(codespatientsinfo[j][0:code_num])
                    else:
                        start=k*codes_num[j][k-1]
                        code.extend(patient_code[j][start :start+code_num])
                        code_date.extend(code_delt_dates[j][start:start+code_num])
                        code_info.extend(codespatientsinfo[j][start:start+code_num])
                elif codes_num[j][k] < code_num:
                    if k==0:
                        code.extend(patient_code[j][:codes_num[j][k]])
                        code.extend([0]*(code_num-codes_num[j][k]))
                        code_date.extend(code_delt_dates[j][:codes_num[j][k]])
                        code_date.extend([0]*(code_num-codes_num[j][k]))
                        code_info.extend(codespatientsinfo[j][:codes_num[j][k]])
                        code_info.extend([[0,0]]*(code_num-codes_num[j][k]))
                    else:
                        start2=0
                        for n in range(k):
                            start2+=codes_num[j][n]
                        code.extend(patient_code[j][start2: start2+codes_num[j][k]])
                        code.extend([0]*(code_num-codes_num[j][k]))
                        code_date.extend(code_delt_dates[j][start2:start2+codes_num[j][k]])
                        code_date.extend([0]*(code_num-codes_num[j][k]))
                        code_info.extend(codespatientsinfo[j][start2:start2+codes_num[j][k]])
                        code_info.extend([[0,0]]*(code_num-codes_num[j][k]))
                else:
                    code.extend(patient_code[j])
                    code_date.extend(code_delt_dates[j])
                    code_info.extend(codespatientsinfo[j])
            batch_patient_code.append(code)
            batch_code_delt_dates.append(code_date)  
            batch_codespatientsinfo.append(code_info)
            # 获取最后一个visit作为预测任务
            f=visit_num-1
            batch_lable.append(patient_visit_code[j][f]) #lable

            # muti-hot lable
    for labeli in batch_lable:
        lable_multi_hot=[0]*icd9_num
        for j in labeli:
            lable_multi_hot[j]=1.0
        batch_lable_multi_hot.append(lable_multi_hot) 
    
        
        # 考虑随机生成的时候可能有的数据始终选不到，因此还要有一个顺序生成
    return batch_patient_code, batch_codespatientsinfo, batch_code_delt_dates,batch_lable_multi_hot

In [4]:
def patient_code_pre_split(outFile,DATA_file):
    # 数据集获取
    patient_code_file = open(outFile + '/patient_code' + '.seqs', 'rb')
    patient_code = pickle.load(patient_code_file)

    code_delt_dates_file = open(outFile + '/code_delt_dates' + '.seqs', 'rb')
    code_delt_dates = pickle.load(code_delt_dates_file)

    visit_delt_dates_file = open(outFile + '/visit_delt_dates' + '.seqs', 'rb')
    visit_delt_dates = pickle.load(visit_delt_dates_file)

    visits_num_file = open(outFile + '/visits_num' + '.seqs', 'rb')
    visits_num = pickle.load(visits_num_file)

    codes_num_file = open(outFile + '/codes_num' + '.seqs', 'rb')
    codes_num = pickle.load(codes_num_file)

    all_codes_num_file = open(outFile + '/all_codes_num' + '.seqs', 'rb')
    all_codes_num = pickle.load(all_codes_num_file)

    visitspatientsinfo_file = open(outFile + '/visitspatientsinfo' + '.seqs', 'rb')
    visitspatientsinfo = pickle.load(visitspatientsinfo_file)

    patient_code_part1=[]
    patient_code_part2=[]
    codes_num_part1=[]
    codes_num_part2=[]
    visits_num_part1=[]
    visit_delt_dates_part1=[]
    visitspatientsinfo_part1=[]
    code_delt_dates_part1=[]
    
    for i in range(len(codes_num)):
        patient_code_part1.append(patient_code[i][:(all_codes_num[i]-codes_num[i][-1])])
        patient_code_part2.append(patient_code[i][all_codes_num[i] - (codes_num[i][-1]):])
        code_delt_dates_part1.append(code_delt_dates[i][:(all_codes_num[i]-codes_num[i][-1])])
        visits_num_part1.append([visits_num[i][0]-1])
        visit_delt_dates_part1.append(visit_delt_dates[i][:-1])
        visitspatientsinfo_part1.append(visitspatientsinfo[i][:-1])
        codes_num_part1.append(codes_num[i][:-1])
        codes_num_part2.append(codes_num[i][-1])
        
    # lable的one-hot
    lables_multi_hot=[]
    for labeli in patient_code_part2:
        lable_multi_hot=[0]*icd9_num
        for j in labeli:
            lable_multi_hot[j]=1
        lables_multi_hot.append(lable_multi_hot)     
    
    # 保存数据
    pickle.dump(patient_code_part1, open(DATA_file + '/patient_code_part1' + '.seqs', 'wb'), -1) 
    pickle.dump(patient_code_part2, open(DATA_file + '/patient_code_part2' + '.seqs', 'wb'), -1) 
    pickle.dump(codes_num_part1, open(DATA_file + '/codes_num_part1' + '.seqs', 'wb'), -1) 
    pickle.dump(codes_num_part2, open(DATA_file + '/codes_num_part2' + '.seqs', 'wb'), -1) 
    pickle.dump(lables_multi_hot, open(DATA_file + '/lables_multi_hot' + '.seqs', 'wb'), -1) 
    pickle.dump(code_delt_dates_part1, open(DATA_file + '/code_delt_dates_part1' + '.seqs', 'wb'), -1) 
    pickle.dump(visits_num_part1, open(DATA_file + '/visits_num_part1' + '.seqs', 'wb'), -1) 
    pickle.dump(visit_delt_dates_part1, open(DATA_file + '/visit_delt_dates_part1' + '.seqs', 'wb'), -1) 
    pickle.dump(visitspatientsinfo_part1, open(DATA_file + '/visitspatientsinfo_part1' + '.seqs', 'wb'), -1) 
    

    return patient_code_part1,patient_code_part2,codes_num_part1,codes_num_part2,lables_multi_hot,code_delt_dates_part1,visits_num_part1,visit_delt_dates_part1,visitspatientsinfo_part1
    

In [5]:
# 训练参数
learning_rate = 1e-3
iters = 20
dropout=1.0

# 网络参数
info_dim=2
icd9_num=259
one_hot_input_dim=icd9_num # 数据len(types)=259, one-hot的长度应该是259
input_dim = 200 
hidden_dim1 =150
output_dim1=150
hidden_dim2=150
output_dim2=150
vhidden_dim=100
voutput_dim=150
output_dim=200

# 生成batch数据参数
outFile='SEQ'
checkpoint_dir_pre='MODEL_pre'
result_dir='RESULT'
batch_size=4
visit_num=2 #visit_num=random.randint() 
code_num=10 #code_num=random.randint() # code_num=visit_length
patient_num=14 #病人的总个数

In [6]:
#把数据集中一个病人的visit分开，用来预测
patient_code_part1,patient_code_part2,codes_num_part1,codes_num_part2,lables_multi_hot,code_delt_dates_part1,visits_num_part1,visit_delt_dates_part1,visitspatientsinfo_part1=patient_code_pre_split('SEQ','PRE_SEQ')


In [7]:
# 实例化
mtgruprecode=MTGRU_precode(visit_num-1, code_num,one_hot_input_dim,input_dim,info_dim,output_dim1,output_dim2,voutput_dim,output_dim,vhidden_dim, hidden_dim1, hidden_dim2,  visits_num_part1,codes_num_part1,visit_delt_dates_part1,visitspatientsinfo_part1,patient_num,patient_code_part1,code_delt_dates_part1)


In [8]:
# 目标
loss,dic = mtgruprecode.get_cross_loss()
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


#### 多次走iters

In [9]:
init = tf.global_variables_initializer()
sess= tf.Session() 
sess.run(init)
Loss = np.zeros(iters)
Acc=np.zeros(iters)

In [16]:
# 生成batch训练
for i in range(iters):
    LOSS = 0 #cross-loss
    ACC=0 #对所有标签预测对的准确度
    
    for i in range(10): #学习完整个数据库需要的次数,要按照实际
        # 生成batch
        xindex, info, t,lable_multi_hot = batch_generator(outFile, visit_num, code_num, patient_num,batch_size,icd9_num)
        if len(xindex)==0: continue
        _, L,D = sess.run([optimizer, loss, dic], feed_dict={mtgruprecode.inputindex: xindex, mtgruprecode.time: t, mtgruprecode.info:info,mtgruprecode.lable_multi_hot:lable_multi_hot,mtgruprecode.keep_prob:dropout})
        LOSS += L
        
        T_num=0.0
        for p in D:
            T_num+=1
            for value in p:
                if value>0.5:
                    T_num-=1
                    break
        acc=T_num/len(D)
        ACC+=acc
    Acc[i]=ACC/10    
    
    Loss[i] = LOSS / 10
    
    #print('Loss %f' %(Loss[i]))
    print('Loss, Acc: %f %f'  %(Loss[i], Acc[i]))

Loss, Acc: 0.025130 0.908333
Loss, Acc: 0.014639 0.975000
Loss, Acc: 0.024474 0.908333
Loss, Acc: 0.024726 0.891667
Loss, Acc: 0.010892 0.816667
Loss, Acc: 0.028854 0.758333
Loss, Acc: 0.017366 0.875000
Loss, Acc: 0.023441 0.858333
Loss, Acc: 0.017876 0.858333
Loss, Acc: 0.016964 0.858333
Loss, Acc: 0.012866 0.933333
Loss, Acc: 0.032120 0.741667
Loss, Acc: 0.018755 0.900000
Loss, Acc: 0.023540 0.891667
Loss, Acc: 0.018706 0.833333
Loss, Acc: 0.016990 0.900000
Loss, Acc: 0.010323 0.975000
Loss, Acc: 0.017528 0.941667
Loss, Acc: 0.021634 0.916667
Loss, Acc: 0.014494 0.975000


In [17]:
pres,attention = sess.run(mtgruprecode.get_all_Pre_sigmoid())

Instructions for updating:
Use `tf.cast` instead.


In [19]:
# 总数据集上全部标签预测对的准确度
right_num=0.0
for i in range(patient_num):
    dic=np.abs(lables_multi_hot[i]-pres[i][0])
    right_num+=1
    for d in dic:
        if d>0.5:
            right_num-=1
            break
accuracy=right_num/patient_num
print accuracy

0.5


In [20]:
# 保存模型
saver = tf.train.Saver()
saver.save(sess, checkpoint_dir_pre + '/model.ckpt') 

'MODEL_pre/model.ckpt'

In [20]:
#保存预测
result_file='RESULT_pre'
pickle.dump(pres, open(result_file + '/pres' + '.seqs', 'wb'), -1) # 一个病人所有的code，不区分visit

### 学习判别阈值
##### 对于每一个类来说找到一个阈值使得f2-score最好


In [21]:
result_file='RESULT_pre'
pres_file = open(result_file + '/pres' + '.seqs', 'rb')
pres = pickle.load(pres_file)
DATA_file='PRE_SEQ'
lables_multi_hot_file = open(DATA_file + '/lables_multi_hot' + '.seqs', 'rb')
labels=pickle.load(lables_multi_hot_file)
icd9_num=259
patient_num=14

In [22]:
def learn_threshold(pres,labels):
    threshold=[0.5]*icd9_num
    f2score=[0.0]*icd9_num
    for i in range(icd9_num): #对每个类
        for th in range(1,10): #每个类的可能的threshold
            thres=th/10.0
            TP=0.0
            FP=0.0
            FN=0.0
            TN=0.0
            for j in range(patient_num):
                if labels[j][i] == 1 and pres[j][0][i] > thres: TP+=1 #TP 
                elif labels[j][i] == 0 and pres[j][0][i] > thres: FP+=1 #FP
                #elif labels[j][i] == 0 and pres[j][0][i] < thres: TN+=1 #TN
                elif labels[j][i] == 1 and pres[j][0][i] < thres: FN+=1 #FN
            if TP==0 : 
                f2score[i]=1 
                continue
            prec= TP/(TP+FP)
            recall=TP/(TP+FN)
            score= 5*prec*recall/(4*prec+recall)
            if score > f2score[i]: 
                threshold[i]=thres
                f2score[i]=score
    mean_f2score=np.mean(f2score)
    print mean_f2score
    return threshold

In [23]:
threshold=learn_threshold(pres,labels)

0.9603369634813734


In [24]:
# 在新的阈值下，对于病人的所有预测都正确的准确度
new_right_num=0.0
for i in range(patient_num):
    new_right_num+=1
    for j in range(icd9_num):
        if pres[i][0][j] > threshold[j] and labels [i][j]==1: continue 
        elif pres[i][0][j] < threshold[j] and labels [i][j]==0: continue 
        else: 
            new_right_num-=1
            break
new_accuracy=new_right_num/patient_num
print new_accuracy

0.428571428571


In [26]:
print threshold

[0.6, 0.1, 0.5, 0.1, 0.1, 0.1, 0.5, 0.1, 0.1, 0.5, 0.1, 0.4, 0.1, 0.5, 0.5, 0.1, 0.1, 0.1, 0.5, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.4, 0.6, 0.1, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.1, 0.5, 0.1, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.1, 0.7, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.7, 0.7, 0.7, 0.7, 0.7, 0.1, 0.7, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.5, 0.5, 0.5, 0.1, 0.1, 0.5, 0.5, 0.1, 0.1, 0.1, 0.1,

In [29]:
# 在新的阈值下，对于病人的所有预测容错率n的准确度
new_right_num2=0.0
for i in range(patient_num):
    n=0
    new_right_num2+=1
    for j in range(icd9_num):
        if pres[i][0][j] > threshold[j] and labels [i][j]==1: continue 
        elif pres[i][0][j] < threshold[j] and labels [i][j]==0: continue 
        elif n!=0: n-=1
        elif n==0: 
            new_right_num2-=1
            break
new_accuracy2=new_right_num2/patient_num
print new_accuracy2

0.428571428571


In [30]:
# 容错
34.0/259

0.13127413127413126

### code的长期影响，从code到visit的attention的值得到

In [55]:
# 每个病人的每个visit下最高贡献的code，死亡病人每个visit下最高贡献code
def PatientImpcode(codeAttention):
    codes_num_file= open(outFile + '/codes_num' + '.seqs', 'rb')
    codes_num=pickle.load(codes_num_file) 
    newSeqs_file= open(outFile + '/newSeqs' + '.seqs', 'rb')
    newSeqs=pickle.load(newSeqs_file) 
    types_file= open(outFile + '/types2' + '.seqs', 'rb')
    types=pickle.load(types_file) 
    type_pids_file= open(outFile + '/type_pids2' + '.seqs', 'rb')
    type_pids=pickle.load(type_pids_file)
    death_labels_file=open(outFile + '/death_labels' + '.seqs','rb')
    death_labels=pickle.load(death_labels_file)
    visits_num_file=open(outFile + '/visits_num' + '.seqs','rb')
    patient_code_file=open(outFile + '/patient_code' + '.seqs','rb')
    patient_code=pickle.load(patient_code_file)
    visits_num=pickle.load(visits_num_file)
    newSeqs_file.close()
    types_file.close()
    type_pids_file.close()
    codes_num_file.close()
    death_labels_file.close()
    visits_num_file.close()
    patient_code_file.close()

    #获取每个patient下的code的attention,每个visit的attention
    code_attentions_value=[]
    for i in range(patient_num):
        code_attention_value=[]
        visit_attention_value=[]
        n=0
        for j in range(len(codes_num[i])-1):
            av=[]
            for k in range(codes_num[i][j]):
                av.append(np.linalg.norm(codeAttention[i][n]))
                n+=1
            code_attention_value.append(av)
        code_attentions_value.append(code_attention_value)
       
    # 对每个patient下的visit下code的attention大小进行排名,visit的attention大小排名
    code_attentions_rank=[]
    for i in range(patient_num):
        code_attention_rank=[]
        for j in range(len(codes_num[i])-1):
            rank=np.argsort(code_attentions_value[i][j])
            if j ==0 : code_attention_rank.append(rank[::-1])
            else:  code_attention_rank.append(rank[::-1]+codes_num[i][j-1])
        code_attentions_rank.append(code_attention_rank)

    # 病人id对应的重要code的icd9编码
    PidImpcodeMap={}
    for i in range(len(code_attentions_rank)):
        a=[]
        impCode=[]
        for j in range(len(code_attentions_rank[i])):
            a.append(types[patient_code[i][code_attentions_rank[i][j][0]]])
        impCode.append(a)
        PidImpcodeMap[type_pids[i]]=impCode
        
    return PidImpcodeMap
    

In [56]:
PidImpcodeMap=PatientImpcode(attention)

In [57]:
# 长期影响的重要code
PidImpcodeMap

{10059: [['D_571.5']],
 10088: [['D_285.9', 'D_276.2']],
 10094: [['D_244.9']],
 10117: [['D_287.5']],
 10119: [['D_276.6']],
 10124: [['D_998.0']],
 40124: [['D_162.8']],
 40310: [['D_453.41']],
 41795: [['D_458.29']],
 41976: [['D_285.9',
   'D_244.9',
   'D_785.52',
   'D_998.11',
   'D_584.9',
   'D_008.45',
   'D_244.9',
   'D_357.2',
   'D_518.81',
   'D_507.0',
   'D_530.19',
   'D_507.0',
   'D_438.82',
   'D_530.19']],
 42135: [['D_572.2']],
 42346: [['D_V10.83']],
 43881: [['D_327.23']],
 44083: [['D_272.4', 'D_V10.03']]}